In [1]:
import numpy as np

In [2]:
# results_array: 4,6,7,8 are the arms, 5 is the middle area
results_array = np.load('results_array.npy')
# results_array[i, j]: how many parts of the frame i are in zone j
results_array

array([[ 0,  0,  0, ..., 14,  7,  0],
       [ 0,  0,  0, ..., 14,  7,  0],
       [ 0,  0,  0, ..., 15,  8,  0],
       ..., 
       [ 0,  0,  0, ...,  0,  9,  0],
       [ 0,  0,  0, ...,  0,  9,  0],
       [ 0,  0,  0, ...,  0,  9,  0]], dtype=int64)

In [3]:
results_array.shape

(9000L, 9L)

In [33]:
#list(set(zip(*results_array)[1]))

In [15]:
zones = {'F1': [(162.31458256068703, 8.2770534550199955), (311.14234344114061, 8.2770534550195975), (312.88827605192603, 164.41331264811259), (158.75509946587033, 164.89430962695039)], 'F2': [(336.98244101442572, 8.2770534550195975), (491.55037290620794, 8.2770534551236778), (491.48469108802612, 163.8559742608403), (338.47580378248279, 164.33346271698855)], 'F3': [(338.70245716665022, 188.01874136248435), (491.48469108802612, 187.54273036968502), (491.4065148076516, 349.03022293373942), (340.21245321108631, 345.81332800605628)], 'F4': [(158.21676409976021, 188.58106573579505), (313.15312463758573, 188.09834330853769), (314.91069100391132, 345.27499263994616), (154.7329938544286, 341.86695653038259)], 'CL': [(158.75509946587033, 164.89430962695039), (312.88827605192603, 164.41331264811259), (313.15312463758573, 188.09834330853769), (158.21676409976021, 188.58106573579505)], 'M': [(312.88827605192603, 164.41331264811259), (338.47580378248279, 164.33346271698855), (338.70245716665022, 188.01874136248435), (313.15312463758573, 188.09834330853769)], 'OB': [(313.15312463758573, 188.09834330853769), (338.70245716665022, 188.01874136248435), (340.21245321108631, 345.81332800605628), (314.91069100391132, 345.27499263994616)], 'CR': [(338.47580378248279, 164.33346271698855), (491.48469108802612, 163.8559742608403), (491.48469108802612, 187.54273036968502), (338.70245716665022, 188.01874136248435)], 'OT': [(311.14234344114061, 8.2770534550195975), (336.98244101442572, 8.2770534550195975), (338.47580378248279, 164.33346271698855), (312.88827605192603, 164.41331264811259)]}

In [16]:
shape = (360, 640)

In [17]:
from PIL import Image,ImageFilter,ImageDraw

In [24]:
def line_fx_from_pts(p1,p2):
    '''returns m and b for a line through two points supplied'''
    run,rise = np.array(p1)-np.array(p2)
    if run == 0:
        run = 10e-3
    m = rise/run
    b = p1[1]-(m*p1[0])
    return m,b

In [25]:
def outline_from_polygon(poly):
    pd_poly = []
    for itr in range(len(poly)):
        i = itr-1
        m,b = line_fx_from_pts(map(float,poly[i]),poly[i+1])
        step = poly[i][0] < poly[i+1][0] and 1 or -1
        edge = [(x,m*x+b) for x in xrange(int(poly[i][0]),int(poly[i+1][0]),step)]
        pd_poly.extend(edge)

    return pd_poly

In [26]:
def mask_from_outline(outline,shape):
    if len(outline) < 2:
        return numpy.zeros(shape,dtype=bool)

    img = Image.new('L', tuple(reversed(shape)), 0)
    ImageDraw.Draw(img).polygon(outline, outline=1, fill=1)
    return np.array(img).astype(bool)

In [27]:
zones_masks = dict([(k, mask_from_outline(outline_from_polygon(v), shape)) for k, v in zones.items()]) 
zones_masks

{'CL': array([[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ..., 
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]], dtype=bool),
 'CR': array([[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ..., 
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]], dtype=bool),
 'F1': array([[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ..., 
        [False, False, False, ..., False, False, False],


In [29]:
zones_order = zones_masks.keys()
zones_order

['F1', 'F2', 'F3', 'F4', 'CL', 'M', 'OB', 'CR', 'OT']

In [35]:
def arm_entry(results_array, zones_order):
    # Arm entries version from 20130829 10:10 am. Doesn't capture extra events anymore. Seems accurate now.
    # each list of arm entries contains when a mouse entered the arm
    arm_entries = [[], [], [], [], [], [], [], [], []] # the number of lists = the number of zones
    for frame in range(len(results_array) - 1): # len(results_array)=9000
        # Compare each arm, one by one to the middle
        for zone in [4, 6, 7, 8]:  # 4,6,7,8 are the arms
            if results_array[
                frame, 5] > 0:  # 5 is the middle area. Arm antries can only occur if a mouse was present in the middle.
                # Arm entry: If in a given frame there is less of a mouse in an arm than in the middle and in the next frame most of the mouse is in an arm
                if results_array[frame, zone] <= results_array[frame, 5] and np.argmax(
                        results_array[frame + 1]) == zone:
                    arm_entries[zone].append(frame + 1)
    tot_arm_entries = dict(zip(zones_order, [len(arm) for arm in arm_entries])) # how many times the mouse entered each arm
    return arm_entries, tot_arm_entries

arm 4: CL  
arm 6: OB  
arm 7: CR  
arm 8: OT

In [38]:
arm_entries, tot_arm_entries = arm_entry(results_array, zones_order)

In [80]:
def turning_preference(arm_entries):
    num_left = 0
    num_right = 0
    num_straight = 0
    num_back = 0
    
    entries = sorted([entry for zones in arm_entries for entry in zones])
    arm_entries_tuple = [(entry, zone) for zone, subentries in enumerate(arm_entries) for entry in subentries]
    entries_dict = dict(arm_entries_tuple)
    
    turn_left = [(4, 8), (8, 7), (7, 6), (6, 4)]
    turn_right = [(8, 4), (7, 8), (6, 7), (4, 6)]
    go_straight = [(4, 7), (7, 4), (6, 8), (8, 6)]
    go_back = [(4, 4), (6, 6), (7, 7), (8, 8)]

    for i in range(len(entries)-1):
        if (entries_dict[entries[i]], entries_dict[entries[i+1]]) in turn_left:
            num_left += 1
        elif (entries_dict[entries[i]], entries_dict[entries[i+1]]) in turn_right:
            num_right += 1
        elif (entries_dict[entries[i]], entries_dict[entries[i+1]]) in go_straight:
            num_straight += 1
        elif (entries_dict[entries[i]], entries_dict[entries[i+1]]) in go_back:
            num_back += 1
            
    return num_left, num_right, num_straight, num_back

In [81]:
turning_preference(arm_entries)

(3, 7, 2, 11)

In [ ]:
'''
    Time: %T_Inside, %T_Outside, %T_Quadrant, %T_safety, %T_peeking, %T(Active)
    Position: Avg(Position in each quadrant), Avg(Manhattan Distance) from safe spot
    Velocity: In each region, Region -> Region
    Preference of Left turns vs Right turns
    Region-> Region frequency
    Feature changes with time
'''